<a href="https://colab.research.google.com/github/ravibalebilalu/ravibalebilalu/blob/main/one_hot_embedding_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data=pd.read_csv("/content/FA-KES-Dataset.csv",encoding='latin')
data.head()

In [ ]:
data.columns

In [ ]:
data=data[['article_content','labels']]
data.head()

In [ ]:
data.info()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
ps=PorterStemmer()
stopWords=set(stopwords.words('english'))



In [ ]:
def clean(text):
  result=re.sub('[^a-zA-Z]',' ',text)
  result=result.lower()
  result=result.split()
  result=[ps.stem(i) for i in result if i not in stopWords]
  result=' '.join(result)
  return result
data['txt']=data["article_content"].apply(clean)
data.txt
 

In [ ]:
voc_size=5000
one_hot_rpt=[one_hot(words,voc_size) for words in data['txt']]
one_hot_rpt[0]

In [ ]:
sent_len=25
emb_doc=pad_sequences(one_hot_rpt,padding='post',maxlen=sent_len)
emb_doc[0]

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])


In [ ]:
import numpy as np
x=np.array(emb_doc)
y=np.array(data['labels'])
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=5)
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test),batch_size=10)
